In [38]:
from src.constants import places
import requests
import pprint
import re
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import os
import json

In [40]:
from pymongo import MongoClient

client = MongoClient()

def connectCollection(database, collection):
    ''' return collection from db crunchbase'''
    db = client[database]
    coll = db[collection]
    return db, coll

In [41]:
db, coll = connectCollection('dbcompanies','companies_clean')

In [42]:
companies = db.companies_clean.find({ 
    "money_raised":{
        "$gte": 1000000
    }}) 

In [43]:
data_companies = pd.DataFrame(companies)

In [44]:
geo_point = []
for loc in range(len(data_companies)):
    geo_point.append(data_companies.coordinates[loc])  

In [45]:
def near(geo_point, radio_meters):
        geopoint = geo_point
        return list(db.companies_clean.find({
        "coordinates.coordinates": {
         "$near": {
             "$geometry": geopoint,
             "$maxDistance": radio_meters
         }}})) 

radio_meters = 5000
list_number_offices = []
list_offices=[]
for i in range(len(data_companies)):
    num_offices =  near(geo_point[i], radio_meters)
    list_offices.append(num_offices)
    list_number_offices.append(len(num_offices))
data_companies['number offices near'] = list_number_offices

In [46]:
data_companies = data_companies[data_companies['number offices near'] > 1]

In [47]:
data_companies.shape

(26, 13)

In [11]:
geo = []
for coords in data_companies['coordinates']:
    geo.append(coords['coordinates'])

In [12]:
def getNearby(lat,lng,query,radius):
        CLIENT_ID = os.getenv("client_id")
        CLIENT_SECRET = os.getenv("client_secret")
        url = 'https://api.foursquare.com/v2/venues/explore'
        queryParams = dict(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET,
            v='20181023', # DONDE ESTOY
            ll=f"{lat},{lng}", #QUE QUIERO
            query=query, # A QUE DISTANCIA
            radius=radius
        )
        res = requests.get(url, params=queryParams)
        return res.json()
  

In [13]:
for coord in geo:
    for place, value in places.items():
        try:
            request = (getNearby(coord[1],coord[0],value['api'],value['distance']))
        except:
            Exception: "Ha habido problemas con este {}".formate(place)
        

In [20]:
def curateResponse(data):
    places = []
    for item in data["response"]["groups"][0]["items"]:
        places.append({
            "type": item["venue"]['categories'][0]['name'],
            "name":item["venue"]['name'],
            "address":item["venue"]['location'],
            "distance": item["venue"]['location']["distance"],
            "coordinates": {
            "type":"Point",
            "coordinates":[item["venue"]['location']["lng"], item["venue"]['location']["lat"]]
                }})
    return places

In [21]:
df = pd.DataFrame(curateResponse(request))

In [26]:
with open ('requests.txt','r+')as f:
    data =  f.readlines()

In [36]:
len(data)

145